In [ ]:
#
# LLM with QnA and Google Search
#
#
!pip install youtube-transcript-api # YouTube lib to get transcript
!pip install pytube                 # YouTube lib
!pip install openai                 # OpenAI GPT
!pip install transformers           # Used to know the token size
!pip install chromadb
!pip install python-dotenv

In [ ]:
!git clone https://github.com/hwchase17/langchain.git

In [31]:
from dotenv import dotenv_values
import os

api_keys = dotenv_values('keys.txt')
os.environ['OPENAI_API_KEY'] = api_keys['OPENAI_API_KEY']
os.environ['SERPER_API_KEY'] = api_keys['SERPER_API_KEY']

In [8]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.7, openai_api_key=api_keys['OPENAI_API_KEY'])

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [18]:
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.document_loaders import YoutubeLoader

In [19]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=FtyKgybYkaE", add_video_info=False)
result = loader.load()

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

docs = text_splitter.split_documents(result)

In [22]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [23]:
embeddings = OpenAIEmbeddings()

In [24]:
db = Chroma.from_documents(docs, embeddings, persist_directory='db')

Running Chroma using direct local API.
loaded in 15 embeddings
loaded in 1 collections
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [25]:
query = "When is the next Devoxx Belgium? Answer truthfully and when you don't know the answer say : 'I don't know'"

In [27]:
docs = db.similarity_search(query)
docs

[Document(page_content="perspective so they rescheduled it to the 23rd of May 2023 so if you are from from Belgium it's I mean Vox day is for the people who don't know this these are like the incubators for devox so there's basically an event from java user groups who organize a one day or a two-day event and a few of those their aim is to become a devoxx now of course here it's a bit of a special situation because we already have a devops um but yeah it's it's uh it will be in in Brussels brussels.voxdays.com and the ticket sales already open and it is a one-day event to just get together and at least you know we sometimes get some I call it the devox blues so like we work nine months to this event and then on Monday I more or less fall in a depression for a week or so because you just missed this energy Etc so luckily in May I can just get it up again and then go back and then socialize so I'm looking forward to that so staying all best best of luck with organizing this and then next

In [28]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)

' The next Devoxx Belgium is from the 2nd to the 6th of October 2021.'

In [32]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

tools = load_tools(["google-serper"], llm=llm)

In [29]:
query = "When is the next Devoxx Belgium?"

In [33]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run(query)



> Entering new AgentExecutor chain...
 I need to find out when this event is happening
Action: Serper Search
Action Input: "Devoxx Belgium 2021"
Observation: Devoxx: Event management company in Antwerp, Belgium. Devoxx Address: Groenendaallaan 394, 2030 Antwerpen, Belgium. Devoxx Hours: Open ⋅ Closes 8 PM. Devoxx Phone: +32 50 67 57 91. Devoxx is a conference by developers, for developers. It is offered at an affordable price so that every developer can enjoy the experience. 2022 Schedule ; Registration & Breakfast. 08:30 09:30 ; Lunch. 12:30 13:30 ; Break. 16:30 16:50 ; Break. 17:20 17:35 ; Break. 18:05 18:20. Devoxx are annual technology conferences organised by developers for developers around the world. More About Us. Filter by Category. Devoxx Events. Devoxx Belgium 2022 AI trailer · Welcome to 19th edition of Devoxx Belgium by Stephan Janssen · Artificial Intelligence: You Are Here by Alan D Thompson · Java *is ... We'll decide end of august if a Devoxx Belgium 2021 edition wil

'Devoxx Belgium 2021 is likely to take place as a hybrid event with CFP and registrations opening in September.'

In [34]:
query = "When is Devoxx Belgium in 2023?"
agent.run(query)



> Entering new AgentExecutor chain...
 I should find a source with reliable information.
Action: Serper Search
Action Input: Devoxx Belgium 2023
Observation: 20th edition : 2 - 6 October 2023. Devoxx Belgium. The 19th edition of Devoxx Belgium, a 5-days technology conference, saw over 3,200 attendees. CFP will open in June and run until mid July 2023. Registrations. UPDATE: We'll release two batches of tickets, one on August 16th at 9h00 AM CEST and another ... Devoxx ; 31 May - 2 Jun 2023. Devoxx Poland 2023 ; 2 - 6 Oct 2023. Devoxx Belgium 2023 ; 10 - 12 Oct 2023. Devoxx Morocco 2023. 2023 edition of Devoxx Belgium will be held at Kinepolis Antwerpen, Antwerp starting on 02nd October. It is a 5 day event organised by The Java Community ... Devoxx Belgium 2023 will take place from October 2nd to the 6th in Antwerp. Science & Technology ... Voxxed Days Brussels 2023 ... Voxxed Days, a spin-off of Devoxx, is coming to Brussels! Yes, you're reading it right, we're talking about a big i

'Devoxx Belgium 2023 will take place from October 2nd to the 6th in Antwerp.'